In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [18]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 4))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 4))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [6]:
garbageCollectionDate[19]

'YarriambiackShireCouncil-garbage-collection-zones.json'

In [7]:
readInJson(garbageCollectionDate[19]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_e7b72b97_8046_4d84_ab9f_874a549f907d.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r..."
1,Feature,ckan_e7b72b97_8046_4d84_ab9f_874a549f907d.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'id': 3, 'route': 'Yarriambiack - Wednesday',..."


In [19]:
df = readInJson(garbageCollectionDate[19])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 4))
df['long'] = df.geo.apply(lambda x: round(x[1], 4))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [20]:
df

,properties,lat,long
0,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r...",142.4084,-36.2634
0,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r...",142.4099,-36.2630
0,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r...",142.4105,-36.2628
0,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r...",142.4108,-36.2632
0,"{'id': 1, 'route': 'Yarriambiack - Monday', 'r...",142.4108,-36.2633
...,...,...,...
4,"{'id': 2, 'route': 'Yarriambiack - Tuesday', '...",142.3933,-36.2484
4,"{'id': 2, 'route': 'Yarriambiack - Tuesday', '...",142.3996,-36.2482
4,"{'id': 2, 'route': 'Yarriambiack - Tuesday', '...",142.4025,-36.2526
4,"{'id': 2, 'route': 'Yarriambiack - Tuesday', '...",142.4061,-36.2575


In [21]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_week']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay == None:
        rub_colDay = ''
    elif rub_colFreq == None: 
        rub_colFreq = ''

    return rub_colFreq + rub_colDay.capitalize()

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))

df.drop('properties', axis = 1, inplace = True)

In [22]:
lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]

,locality,long,lat,landfill_colDay,recyclable_colDay


In [27]:
yarriambiack = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'landfill_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'landfill', 'recyclable'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

yarriambiack.to_csv('../GBD_yarriambiack.csv', index = False)

In [189]:
insertQuery = 'INSERT INTO wasteCollection(binInfo_bin_type, postcode, collection_time) VALUES\n'
values = ''

for i in _.index: 
    values = values + f'("{_.bin_type[i].strip()}", "{_.postcode[i].strip()}", "{_.collection_time[i].strip()}"),\n'.replace('"', "'")
    
    

In [191]:
print(insertQuery + values)

INSERT INTO wasteCollection(binInfo_bin_type, postcode, collection_time) VALUES
('landfill', '3371', 'Every Second Thursday'),
('landfill', '3143', 'Every Tuesday'),
('landfill', '3183', 'Every Tuesday'),
('landfill', '3463', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Wednesday'),
('landfill', '3465', 'Every Second Wednesday'),
('landfill', '3186', 'Every Wednesday'),
('landfill', '3472', 'Every Second Monday'),
('landfill', '3464', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Monday'),
('landfill', '3161', 'Every Tuesday'),
('landfill', '3465', 'Every Second Tuesday'),
('landfill', '3185', 'Every Wednesday'),
('landfill', '3184', 'Every Wednesday'),
('landfill', '3695', 'Every Second Wednesday'),
('landfill', '3688', 'Every Second Friday'),
('landfill', '3688', 'Every Second Monday'),
('landfill', '3688', 'Every Second Tuesday'),
('landfill', '3364', 'Every Second Wednesday'),
('landfill', '3691', 'Every Second Tuesday'),
('landfill', '8001', 'Every

In [135]:
df = pd.read_json('GarbageCollectionDate/FrankstonCityCouncil-garbage-collection-zones.json')

In [136]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])


In [139]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])
    
for colFreq in ['landfill_colFreq', 'recyclable_colFreq']: 
    collectionDate_geo[colFreq] = collectionDate_geo[colFreq].apply(lambda x: 'Every Second ' if x == '2' else 'Every ')
    

In [143]:
collectionDate_geo

,properties,geo,landfill_colDay,landfill_colFreq,recyclable_colDay,recyclable_colFreq
0,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[[145.205084864, -38.1446568331], [145.2283478...",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1441279116, -38.1474438406]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1447778394, -38.1477778302]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.144971836, -38.1479238793]",Every Tuesday,Every,Every Tuesday,Every
1,"{'rub_day': 'Tuesday', 'rub_weeks': 1, 'rub_st...","[145.1450769143, -38.1480438368]",Every Tuesday,Every,Every Tuesday,Every
...,...,...,...,...,...,...
5,"{'rub_day': 'Thursday', 'rub_weeks': 1, 'rub_s...","[145.118492867, -38.1456328741]",Every Thursday,Every,Every Thursday,Every
6,"{'rub_day': 'Thursday', 'rub_weeks': 1, 'rub_s...","[[145.1328198908, -38.1534268326], [145.133164...",Every Thursday,Every,Every Thursday,Every
7,"{'rub_day': 'Monday', 'rub_weeks': 1, 'rub_sta...","[[145.1572299083, -38.1126808407], [145.160402...",Every Monday,Every,Every Monday,Every
8,"{'rub_day': 'Friday', 'rub_weeks': 1, 'rub_sta...","[[145.1968809036, -38.1024838245], [145.172497...",Every Friday,Every,Every Friday,Every


In [142]:
collectionDate_geo['landfill_colDay'] = collectionDate_geo.landfill_colFreq + collectionDate_geo.landfill_colDay
collectionDate_geo['recyclable_colDay'] = collectionDate_geo.recyclable_colFreq + collectionDate_geo.recyclable_colDay

collectionDate_geo['lat'] = collectionDate_geo.geo.apply(lambda x: round(x[0], 6))
collectionDate_geo['long'] = collectionDate_geo.geo.apply(lambda x: round(x[1], 6))

collectionDate_geo = collectionDate_geo[['lat', 'long', 'landfill_colDay', 'recyclable_colDay']]

tmp = lonlatSuburb.merge(collectionDate_geo)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay'])\
.sort_values(['locality'])

TypeError: type list doesn't define __round__ method

In [78]:
df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])
collectionDate_geo = df[['properties', 'geo']].explode('geo')

newCols = ['landfill_colDay', 'landfill_colFreq', 'recyclable_colDay', 'recyclable_colFreq']
newColsKeys = ['RUB_DAY', 'RUB_WEEKS', 'REC_DAY', 'REC_WEEKS']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    try: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val])
    except: 
        collectionDate_geo[key] = collectionDate_geo.properties.apply(lambda x: x[val.lower()])
    
for colFreq in ['landfill_colFreq', 'recyclable_colFreq']: 
    collectionDate_geo[colFreq] = collectionDate_geo[colFreq].apply(lambda x: 'Every Second ' if x == '2' else 'Every ')
    
collectionDate_geo['landfill_colDay'] = collectionDate_geo.landfill_colFreq + collectionDate_geo.landfill_colDay
collectionDate_geo['recyclable_colDay'] = collectionDate_geo.recyclable_colFreq + collectionDate_geo.recyclable_colDay

collectionDate_geo['lat'] = collectionDate_geo.geo.apply(lambda x: round(x[0], 6))
collectionDate_geo['long'] = collectionDate_geo.geo.apply(lambda x: round(x[1], 6))

collectionDate_geo = collectionDate_geo[['lat', 'long', 'landfill_colDay', 'recyclable_colDay']]

tmp = lonlatSuburb.merge(collectionDate_geo)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay'])\
.sort_values(['locality'])

type1CollectionDates = pd.concat([type1CollectionDates, tmp])